In [ ]:
import xml.etree.ElementTree as ET
import os
import pandas as pd
from PIL import Image
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
train_image_path = '/content/gdrive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/Classification_Train'
train_classes = []
train_img_per_class = []
train_image_path_lst = sorted(os.listdir(train_image_path))
for folder in train_image_path_lst:
  train_classes.append(folder)
  train_img_per_class.append(len(os.listdir(f'{train_image_path}/{folder}')))
num_classes = len(train_classes)

In [ ]:
# 1) making csv files - coin and dish coordinate
quantity = ['Q1','Q2','Q3', 'Q4', 'Q5']
train_xml_path = '/content/gdrive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/원천/[라벨]양추정_TRAIN/xml' 
xml_array = []
for i in train_classes: 
  %cd /
  path = train_xml_path + '/' +i
  xml_for_class = [] 
  %cd $path
  for j in quantity:
    quantity_path = path[:] + '/' + j
    xml_lst = sorted(os.listdir(quantity_path))[1:] # 0번째는 압축파일
    for xml_file in xml_lst:
      xml_file_name = quantity_path + '/' + xml_file
      image_name = xml_file[:-4] + '.JPG'
      tree = ET.parse(xml_file_name)
      root = tree.getroot()
      unit_lst = []
      obj = root.findall('object')
      dish = obj[4].find('bndbox')
      xmin = int(dish.find('xmin').text)
      ymin = int(dish.find('ymin').text)
      xmax = int(dish.find('xmax').text)
      ymax = int(dish.find('ymax').text)
      dish_xy = [image_name, xmin, ymin, xmax, ymax]
      unit_lst.extend(dish_xy)
      xml_for_class.append(unit_lst)
  xml_array.append(xml_for_class) # Q1(정렬) Q2(정렬) .. 

In [ ]:
# 2) making csv files - coin and dish coordinate using pandas
%cd /content/gdrive/MyDrive/food_csv
for i in range(len(xml_array)):
  cooridate_file = pd.DataFrame(xml_array[i])
  cooridate_file.to_csv( f'{train_classes[i]}.csv', index = False)

/content/gdrive/MyDrive/food_csv


In [ ]:
# make folder for crop
train_crop_image_path = '/content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/train_cropped'
%cd /content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/train_cropped
for i in train_classes:
  folder_name = f'{train_crop_image_path}/{i}'
  os.mkdir(folder_name)

In [ ]:
# path
train_image_path = '/content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/Classification_Train'
coordinate_path = '/content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/train_coordinate'

In [ ]:
# crop all image with same name
%cd /content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/Classification_Train
for food in train_classes:
  image_path_folder = f'{train_image_path}/{food}' # folder name
  %cd $food
  image_file_list = sorted(os.listdir(image_path_folder)) # image name 
  list.sort(image_file_list, key=lambda x: (x[-12:-10])) # 재정렬
  coordinate_path_file = f'{coordinate_path}/{food}.csv' # file name
  coordinate_path_pandas = pd.read_csv(coordinate_path_file)
  xy_coordinate_pandas = coordinate_path_pandas.iloc[:, 4:]

  for j in range(len(image_file_list)):
    image = Image.open(image_file_list[j])
    image = image.crop(xy_coordinate_pandas.iloc[j])
    image.save(f'/content/drive/MyDrive/DCD_2022/음식 이미지 및 영양정보 텍스트/train_cropped/{food}/{image_file_list[j]}')
  %cd ..